# Openai whisper Template (GPU)

Das ist ein kleines Notebook, was einem hilft auf einer GPU eine Transkription einer Audio-Datei zu erstellen mit dem openai whisper package.
Im Grunde muss nur der Name der Datei ausgetauscht werden. 
<br>
Whisper: https://github.com/openai/whisper

In [1]:
!pip install -U openai-whisper

In [2]:
!pip install ffmpeg-python #Eine notwendige Abhängigkeit, damit whisper funktioniert


In [3]:
!pip install torch torchvision torchaudio #


## Belegung der Grafikkarte

Hinweis: Das meiste hier ist optional. Es ist aber immer sinnvoll zu sehen, wie hoch die aktuelle Auslastung der GPU ist <br>

In [4]:
import torch
import torchvision

print(f"PyTorch CUDA Version: {torch.version.cuda}")
print(f"torchvision CUDA Version: {torchvision.__version__}")

PyTorch CUDA Version: 12.4
torchvision CUDA Version: 0.20.0+cu124


In [5]:
!nvidia-smi

Tue Oct 22 18:41:33 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:4B:00.0 Off |                  N/A |
| 30%   42C    P8    20W / 250W |   1041MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
import torch

# Speicherinformationen abrufen
gpu_memory_allocated = torch.cuda.memory_allocated() / 1024**3  # In GB
gpu_memory_reserved = torch.cuda.memory_reserved() / 1024**3  # In GB
gpu_memory_free = torch.cuda.get_device_properties(0).total_memory / 1024**3 - gpu_memory_reserved  # In GB

print(f"Belegter Speicher: {gpu_memory_allocated:.2f} GB")
print(f"Reservierter Speicher: {gpu_memory_reserved:.2f} GB")
print(f"Freier Speicher: {gpu_memory_free:.2f} GB")


Belegter Speicher: 0.00 GB
Reservierter Speicher: 0.00 GB
Freier Speicher: 10.75 GB


In [7]:
pray_to_cuda = torch.cuda.empty_cache()
print(pray_to_cuda)

None


*Prüfen, ob cuda verfügbar ist um auf der GPU zu rendern*

In [8]:
import torch

# Prüfen, ob CUDA verfügbar ist
print(torch.cuda.is_available())  # Gibt 'True' zurück, wenn eine GPU verfügbar ist

# Prüfen, wie viele GPUs verfügbar sind
print(torch.cuda.device_count())

# Den Namen der GPU abrufen
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))  # 0 ist der Index der ersten GPU


True
1
NVIDIA GeForce RTX 2080 Ti


## Modell

Hinweis 1: Das Test-Audio funktioniert mit allen Modellen von tiny bis medium. <br> Hinweis 2: Es lohnt sich bei einer Videodatei zu testen, vorher die Tonspur zu extrahieren, um schnellere und bessere Ergebnisse zu bekommen. <br>
Hinweis 3: .to(device) legt das Modell auf die GPU. Dazu wie folgend die Variable device definieren. 

In [9]:
import whisper
import torch

# Sicherstellen, dass die GPU verwendet wird, falls verfügbar
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Lade das Whisper-Modell und verschiebe es auf die GPU (falls verfügbar)
model = whisper.load_model("medium").to(device)

/opt/conda/lib/python3.9/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


In [10]:
# load audio and pad/trim it to fit 30 seconds
audio_file = "audio/test.mp3"  #Hier den Dateinamen austauschen! 
audio = whisper.load_audio(audio_file)
audio = whisper.pad_or_trim(audio)

# Prüfen, wie viele Kanäle die Audiodatei hat (wichtig, um zu sehen, ob der Input für das Modell passt)
print(audio.shape)

# Log-Mel-Spektrogramm erstellen
mel = whisper.log_mel_spectrogram(audio).to(device)

# Prüfe die Dimensionen des Mel-Spektrogramms
print(mel.shape)

(480000,)
torch.Size([80, 3000])


*Hinweis*

Mel-Skala: Logarithmische Skala, die die menschliche Wahrnehmung von Tonhöhen nachahmt.

Mel-Spektrogramm: Spektrogramm, das Frequenzen auf der Mel-Skala darstellt, um das menschliche Gehör besser widerzuspiegeln.

Verwendung: Wird in der Sprachverarbeitung und Audioanalyse eingesetzt, um die Audioinformationen so darzustellen, dass sie für maschinelle Lernmodelle besser zugänglich sind.

## Transkript Output

Festlegen der Optionen <br>
Funktion zur Anpassung des Outputs (Timestamps, Speaker etc.) <br>
Speichern des Outputs in einer .txt

In [15]:
# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# Dekodierungsoptionen mit aktiviertem "timestamps"
options = whisper.DecodingOptions(fp16=True, language="de", task="transcribe")

# Dekodiere das Audio mit Timestamps
result = model.transcribe(audio_file, fp16=True)

# Funktion zum Formatieren der Timestamps in Minuten:Sekunden
def format_timestamp(seconds):
    minutes = int(seconds // 60)
    seconds = int(seconds % 60)
    return f"{minutes:02d}:{seconds:02d}"


# Falls es mehr als 2 Sprechende im Audio gibt müssen alle mit "a" gekennzeichneten Zeilen entkommentiert werden

#a Wechselnde Sprecher: "A", "B"
#a speakers = ["A", "B"]
#a current_speaker = 0

# Erkannten Text und Timestamps in Minuten ausgeben
for segment in result['segments']:
    start = format_timestamp(segment['start'])
    end = format_timestamp(segment['end'])
    text = segment['text']
    #a speaker = speakers[current_speaker]
    print(f"[{start} - {end}] {text}")
    
    #a Sprecher wechseln: A -> B -> A -> B usw.
    #a current_speaker = (current_speaker + 1) % 2
    
    #a print(f"[{start} - {end}] {speaker}: {text}")
        
        
# Erkannten Text, Timestamps und Sprecherangaben in eine .txt-Datei speichern
with open("output/transcript_gpu.txt", "w") as file:
    current_speaker = 0  # Setze den Sprecher zurück
    for segment in result['segments']:
        start = format_timestamp(segment['start'])
        end = format_timestamp(segment['end'])
        text = segment['text']
        #a speaker = speakers[current_speaker]

        #a Sprecher wechseln: A -> B -> A -> B
        #a current_speaker = (current_speaker + 1) % 2

        file.write(f"[{start} - {end}] {speaker}: {text}\n")

Detected language: de
[00:00 - 00:06]  Es ist Feierabend in der Krossenkrabbe und Tadeusz beantwortet einen letzten Anruf von einem Kunden.
[00:06 - 00:16]  Gerade als er ihnen sagen will, dass das Restaurant geschlossen ist, erscheint der Chef plötzlich und schnappt sich das Telefon von Tadeusz.
[00:16 - 00:26]  Der Kunde möchte eine Pizza bestellen und geliefert bekommen, eine Bestellung, die er gerne annimmt, da er weiß, dass er zusätzliches Geld verdienen kann.
[00:26 - 00:33]  Tadeusz protestiert damit, indem er sagt, dass das Restaurant keine Pizza liefert.
